# Photon Coefficients Calculator for Photoactive or Prism Reactions   

## **1.** Import the required packages 

In [1]:
# Import library
from scipy.interpolate import interp1d
from pathlib import Path
import csv

## 2. Import The Data in csv File

In [182]:
Here = Path().absolute()
NameDir = '/Light emitted from a source/'
NameFile = NameDir + 'White LED Spectral Irradiance'
Data_File = str(Here) + NameFile + '.csv'
print(Data_File)
typeR = 'PrismReaction'

/mnt/c/users/flron/OneDrive/Tesis/Cobrapy Model Development/Models Test/Model Chlorella Analysis/Light emitted from a source/White LED Spectral Irradiance.csv


##### Creating lists from a .csv file

In [183]:
with open(Data_File, "r", encoding="utf-8") as f:
    lector = csv.reader(f, delimiter=',')
    # Omit header
    next(lector, None)
    # Create a list for each column
    wavelength = []
    flux = []
    for row in lector:
        wavelength.append(float(row[0]))
        #Column in file to take data
        flux.append(float(row[2]))
  

<p style="font-size:10pt">Sorting the data</p> 

In [184]:
def sort_list(list1, list2):
 
    zipped_pairs = zip(list2, list1)
 
    z = [x for _, x in sorted(zipped_pairs)]
     
    return z

flux = sort_list(flux, wavelength)
wavelength = sorted(wavelength)

### **3.** Data interpolation

<p style="font-size:10pt">Complement the wavelenght ranges between 250 to 750 nm with 0 Flux</p> 

In [185]:
steps = 6

if wavelength[0] > 250:
    lowerRange = wavelength[0] - 250
    lowerh = lowerRange/steps
    InitWavelength = [250 + i*lowerh for i in range(steps)]
    InitFlux = [0 for i in range(steps)]
else:
    InitWavelength = []
    InitFlux = []

if wavelength[-1] < 750:
    UpperRange = 750 - wavelength[-1]
    Upperh = UpperRange/steps
    FinalWavelength = [wavelength[-1] + (i+1)*Upperh for i in range(steps)]
    FinalFlux = [0 for i in range(steps)]
else:
    FinalWavelength = []
    FinalFlux = []

wavelength = InitWavelength + wavelength + FinalWavelength
flux = InitFlux + flux + FinalFlux

<p style="font-size:10pt">Find the minimal step as difference among two data on x-axis</p>

In [186]:
# List with wavelength values
DiffList = []

for i in range(0, len(wavelength)-1):
    dif = wavelength[i+1] - wavelength[i]
    DiffList.append(dif)

h = round(min([value for value in DiffList if value!=0]),8)
print(f'The step value is = {h}')

The step value is = 2.0


<p style="font-size:10pt">Generate a new wavelenght vector with a constant step</p>

In [189]:
new_w = [wavelength[0]]

while new_w[-1] < wavelength[-1]:
    w = round(new_w[-1] + h,8)
    if w < wavelength[-1]:
        new_w.append(w)
    else:
        break

<p style="font-size:10pt">Generation of an interpolation vector of respective data</p> 

In [190]:
interpolate = interp1d(wavelength, flux, kind='linear')

# List with interpolated flux values
new_f = interpolate(new_w)

# See interpolated data
for i in range(len(new_w)):
    print(f'wavelength = {new_w[i]}, flux = {new_f[i]}')

wavelength = 250.0, flux = 0.0
wavelength = 252.0, flux = 0.0
wavelength = 254.0, flux = 0.0
wavelength = 256.0, flux = 0.0
wavelength = 258.0, flux = 0.0
wavelength = 260.0, flux = 0.0
wavelength = 262.0, flux = 0.0
wavelength = 264.0, flux = 0.0
wavelength = 266.0, flux = 0.0
wavelength = 268.0, flux = 0.0
wavelength = 270.0, flux = 0.0
wavelength = 272.0, flux = 0.0
wavelength = 274.0, flux = 0.0
wavelength = 276.0, flux = 0.0
wavelength = 278.0, flux = 0.0
wavelength = 280.0, flux = 0.0
wavelength = 282.0, flux = 0.0
wavelength = 284.0, flux = 0.0
wavelength = 286.0, flux = 0.0
wavelength = 288.0, flux = 0.0
wavelength = 290.0, flux = 0.0
wavelength = 292.0, flux = 0.0024734391999998648
wavelength = 294.0, flux = 0.0173140743999999
wavelength = 296.0, flux = 0.032154709599999935
wavelength = 298.0, flux = 0.046995344799999964
wavelength = 300.0, flux = 0.06183598
wavelength = 302.0, flux = 0.055236696
wavelength = 304.0, flux = 0.027370671
wavelength = 306.0, flux = 0.02044635
wave

<p style="font-size:10pt">Print the Interpolated Data</p> 

In [191]:
with open(str(Here) + NameFile + "_Interpolated.csv", "w", encoding="utf-8") as f:
    for w in new_w:
        f.write("%f, %f %s" % (w, new_f[new_w.index(w)],"\n"))

### **4.** Function - Simpson's 3/8 Rule 

<p style="font-size:10pt">Simpson's 3/8 Integration</p> 

In [192]:
def simpson38(x, y):
    # x, list which contains values of the independent variable
    # y, list which contains values of the dependent variable

    # calculating step size
    data_pairs = len(x)-1
    h = (x[-1] - x[0]) / data_pairs
    integration = y[0] + y[-1]
    
    for i in range(1,data_pairs):
        k = x[0] + i*h
        if i%2 == 0 and k in x:
            integration = integration + 2 * round(y[x.index(k)], 8)
        elif i%2 == 0 and k in x == False:
            integration = integration + 2 * round(y[i], 8)
        elif i%2 != 0 and k in x:
            integration = integration + 3 * round(y[x.index(k)], 8)
        else:
            integration = integration + 3 * round(y[i], 8)
    
    # Finding final integration value
    integration = integration * 3 * h / 8
    
    return integration

<p style="font-size:10pt">Defining the Photon vector</p> 

In [193]:
photStep = 30
PhotonVect = {'photon'+str(int(250 + (i+0.5)*photStep)): {"low":250+i*photStep, "up":250+(i+1)*photStep} for i in range(int(500/photStep)-1)}

PhotonVect

{'photon265': {'low': 250, 'up': 280},
 'photon295': {'low': 280, 'up': 310},
 'photon325': {'low': 310, 'up': 340},
 'photon355': {'low': 340, 'up': 370},
 'photon385': {'low': 370, 'up': 400},
 'photon415': {'low': 400, 'up': 430},
 'photon445': {'low': 430, 'up': 460},
 'photon475': {'low': 460, 'up': 490},
 'photon505': {'low': 490, 'up': 520},
 'photon535': {'low': 520, 'up': 550},
 'photon565': {'low': 550, 'up': 580},
 'photon595': {'low': 580, 'up': 610},
 'photon625': {'low': 610, 'up': 640},
 'photon655': {'low': 640, 'up': 670},
 'photon685': {'low': 670, 'up': 700}}

### **5.** Integrate the Results

<p style="font-size:10pt">Finding the Whole Integral</p> 

In [194]:
# Integral value usign Simpson's 3/8 Rule
integral = simpson38(new_w, new_f)
print(f'Simpson 3/8 integral = {integral}')

# Integral value usign Scipy method
from scipy import integrate
integral_2 = integrate.simpson(new_f, new_w)
print(f'scipy integral = {integral_2}')

print ('Error: %f' % (100*abs(integral_2-integral)/integral))

Simpson 3/8 integral = 2614.0154800214996
scipy integral = 2788.2683755743997
Error: 6.666100


<p style="font-size:10pt">Integration by wavelength range</p> 

In [195]:
integralTotal = 0

for j, k in PhotonVect.items():
    low_w = k.get('low')
    high_w= k.get('up')
    
    low_index = 0
    while new_w[low_index] < low_w:
        low_index += 1

    high_index = 0
    while new_w[high_index] < high_w:
        high_index += 1
    
    integralPhoton = simpson38(new_w[low_index:high_index], new_f[low_index:high_index])
    PhotonVect[j] = k | {"Integral":integralPhoton} 
    print(f'Photon vector = {j},  low limit = {low_w}, high limit = {high_w}, Simpson 3/8 integral = {integralPhoton}')
    integralTotal += integralPhoton

print(f'Total integral = {integralTotal}')

Photon vector = photon265,  low limit = 250, high limit = 280, Simpson 3/8 integral = 0.0
Photon vector = photon295,  low limit = 280, high limit = 310, Simpson 3/8 integral = 0.5013896032499999
Photon vector = photon325,  low limit = 310, high limit = 340, Simpson 3/8 integral = 0.82791195525
Photon vector = photon355,  low limit = 340, high limit = 370, Simpson 3/8 integral = 0.581345805
Photon vector = photon385,  low limit = 370, high limit = 400, Simpson 3/8 integral = 0.431651277
Photon vector = photon415,  low limit = 400, high limit = 430, Simpson 3/8 integral = 1.0922873385
Photon vector = photon445,  low limit = 430, high limit = 460, Simpson 3/8 integral = 16.454988630000003
Photon vector = photon475,  low limit = 460, high limit = 490, Simpson 3/8 integral = 93.44190096225
Photon vector = photon505,  low limit = 490, high limit = 520, Simpson 3/8 integral = 147.4865386485
Photon vector = photon535,  low limit = 520, high limit = 550, Simpson 3/8 integral = 279.3441851947499

<p style="font-size:10pt">Getting Stoichiometric coefficients</p> 

In [196]:
coeff = []

if typeR == 'PhotoReaction':
    for j, k in PhotonVect.items():
        PhotoActiveArea = k.get('Integral')
        if PhotoActiveArea != 0:
            coeff.append(integralTotal/PhotoActiveArea)
        else:
            coeff.append(5000)
    
    min_coeff = min([c for c in coeff])
    coeff = [c/min_coeff for c in coeff]
elif typeR == 'PrismReaction':
    for j, k in PhotonVect.items():
        PhotoActiveArea = k.get('Integral')
        coeff.append(PhotoActiveArea/integralTotal)

count = 0

for j, k in PhotonVect.items():
    PhotonVect[j] = k | {"Coefficient":coeff[count]}
    count += 1

PhotonVect

{'photon265': {'low': 250, 'up': 280, 'Integral': 0.0, 'Coefficient': 0.0},
 'photon295': {'low': 280,
  'up': 310,
  'Integral': 0.5013896032499999,
  'Coefficient': 0.00022873209300520322},
 'photon325': {'low': 310,
  'up': 340,
  'Integral': 0.82791195525,
  'Coefficient': 0.0003776903891123169},
 'photon355': {'low': 340,
  'up': 370,
  'Integral': 0.581345805,
  'Coefficient': 0.0002652078181827452},
 'photon385': {'low': 370,
  'up': 400,
  'Integral': 0.431651277,
  'Coefficient': 0.00019691772505172852},
 'photon415': {'low': 400,
  'up': 430,
  'Integral': 1.0922873385,
  'Coefficient': 0.0004982974666381616},
 'photon445': {'low': 430,
  'up': 460,
  'Integral': 16.454988630000003,
  'Coefficient': 0.00750670529528321},
 'photon475': {'low': 460,
  'up': 490,
  'Integral': 93.44190096225,
  'Coefficient': 0.04262785156082185},
 'photon505': {'low': 490,
  'up': 520,
  'Integral': 147.4865386485,
  'Coefficient': 0.06728281650934732},
 'photon535': {'low': 520,
  'up': 550,
 

### **6.** Print the coefficcients for each reaction in a file

In [197]:
Reaction = 'White LED Spectral Irradiance'

with open(str(Here) + NameDir + typeR +" Coefficients.csv", "a", encoding="utf-8") as f:
    for name, Dir in PhotonVect.items():
        low_w = Dir.get('low')
        up_w = Dir.get('up')
        StCoef = Dir.get('Coefficient')
        f.write("%s, %s, %i, %i, %f %s" % (Reaction, name, low_w, up_w, StCoef,"\n"))